In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"


In [2]:

model = transformers.AutoModelForCausalLM.from_pretrained(model_id,
torch_dtype=torch.bfloat16,  device_map="auto")

tokenizer=transformers.AutoTokenizer.from_pretrained(model_id)
pipeline = transformers.pipeline(
    "text-generation", model=model, 
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
#LoRA
from peft import get_peft_model, LoraConfig, TaskType
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                         inference_mode=False, # 学習時はFalse
                         r=32, 
                         lora_alpha=64,
                         lora_dropout=0.05,
                         bias="none",
                         target_modules=["q_proj", "v_proj", "k_proj", "o_proj", 
                                         #"gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"
                                         ],
                         )

# モデルにLoRAアダプター適用、更新対象のパラメータ数の確認
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 131,072,000 || all params: 70,684,778,496 || trainable%: 0.1854


In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
jsonl_path="data/20240919063234_llm_gen.jsonl"
error_threshold=0.1
max_records=20000

df=pd.read_json(jsonl_path,lines=True)
df=df.drop(columns=["record","prompt"])
df=df.sort_values(by="error_rate")
df["cond"]=df["CompoundName"]+" "+df["SMILES"]+" "+df["Property"]
df=df.drop_duplicates(subset=["cond"])
df=df[df["error_rate"]<error_threshold]
df=df[:max_records]
df["q"]="Predict "+df["Property"]+" "+df["Unit"]+" for "+df["CompoundName"]+" (Compound X) with SMILES "+df["SMILES"]+". The prediction consists of #Reason and #Prediction. The #Reason is the quantitative explanation of the prediction. The #Prediction is the predicted value and the unit of the prediction."
df["a"]="#Reason\n"+df["reason"]+"\n#Prediction\n"+df["predicted"].astype(str)+" "+df["Unit"]
q_list=df["q"].tolist()
a_list=df["a"].tolist()
df.shape

(4286, 11)

In [5]:
train_text_list=[]
for i in range(len(q_list)):
    q=q_list[i]
    a=a_list[i]
    messages=[
        {"role": "user", "content": q},
        {"role": "assistant", "content": a}
    ]
    prompt=tokenizer.apply_chat_template(messages,tokenize=False)
    train_text_list.append(prompt)


In [6]:
import datasets
ds=datasets.Dataset.from_list([{"text":t} for t in train_text_list])

In [7]:
from trl import DataCollatorForCompletionOnlyLM

# response_templateは必須指定
response_template = "<|start_header_id|>assistant<|end_header_id|>\n\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['text'])):
        text = f"{example['text'][i]}"
        output_texts.append(text)
    return output_texts

In [8]:
from transformers import TrainingArguments
from trl import SFTTrainer

# SFTTrainerはTrainingArgumentsを使用することができる。
# 指定しない場合、TrainingArgumentsのデフォルトが指定される。
args = TrainingArguments(
    output_dir=f'./output0919_threshold_{error_threshold}',
    num_train_epochs=1,
    gradient_accumulation_steps=16,
    per_device_train_batch_size=8,
    save_strategy="steps",
    save_steps=5,
    logging_steps=1,
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,
    warmup_ratio=0.03,
    weight_decay=0.001,
    learning_rate=1e-4,
    #save_total_limit=1,
    fp16=True,
)


# data_collatorが指定されていない場合、以下のようにDataCollatorForLanguageModelingがmlm=Falseで使われる。
# つまり通常のCausal LMを学習することになる。
# if data_collator is None:
#     data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# packing=False（default）ではdataset_text_fieldかformatting_funcを指定する必要あり
trainer = SFTTrainer(
    lora_model,
    args=args,
    train_dataset=ds,
    formatting_func=formatting_prompts_func,
    max_seq_length=1024,
    data_collator=collator,
)


/home/hatakeyama/miniconda3/envs/llama/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/hatakeyama/miniconda3/envs/llama/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/4286 [00:00<?, ? examples/s]

/home/hatakeyama/miniconda3/envs/llama/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:

tokenizer.pad_token = tokenizer.eos_token

In [10]:
trainer.train()

Step,Training Loss
1,0.739400
2,0.764400
3,0.668300
4,0.658400
5,0.626800
6,0.558500
7,0.506700
8,0.509000
9,0.487300
10,0.498400


TrainOutput(global_step=33, training_loss=0.4906843033703891, metrics={'train_runtime': 3639.4073, 'train_samples_per_second': 1.178, 'train_steps_per_second': 0.009, 'total_flos': 8.854884523223286e+17, 'train_loss': 0.4906843033703891, 'epoch': 0.9850746268656716})

In [11]:

trainer.save_model()